라이브러리 셋팅

In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from tqdm import tqdm
from time import sleep

크롤링한 데이터들을 담을 딕셔너리 생성

In [ ]:
nogada = {
    '사건번호':[],
    '내용' : [],
    '결과' : [],
    '날짜' : [],
    '관할위원회':[]
}

In [ ]:
for page in tqdm(range(1,1680)): #서울 노동위원회 부당해고 데이터 크롤링 - 20분걸림


  url = 'http://www.nlrc.go.kr/nlrc/md/search_case.go'
  headers = {
    'Referer': 'http://www.nlrc.go.kr/nlrc/md/search_case.go',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
}
  data = {
'inputChk': '해고',
'pQuery_tmp': '해고',
'even_gubn':[] ,
'currentPage':str(page) ,
'searchGubn': 'BH',
'pQuery': '해고',
'key_end_fdate':[] ,
'key_end_tdate':[] ,
'key_comm_code': '01'}


# 리퀘스트를 짧은 시간에 여러번 넣으면 접근이 막히는 문제가 발생하기에 except 넣어줌.
  try:
    resp = requests.post(url, headers=headers, data=data)
    soup = BeautifulSoup(resp.content, 'lxml')
  except requests.exceptions.ConnectionError:
    resp.status_code = "Connection refused"

  
  num_result_tags = soup.select('dl span.txt_blue')
  for i in range(10):
    nogada['사건번호'].append(num_result_tags[::2][i].text.replace('|',''))
    nogada['결과'].append(num_result_tags[1::2][i].text.replace('|',''))
    nogada['내용'].append(soup.select('dl dd.text-cut2')[i].text)
    nogada['날짜'].append(soup.select('dl span.search_date')[i].text.strip())
    nogada['관할위원회'].append(soup.select('dl dt a')[i]['onclick'].split("'")[3])

100%|██████████| 1679/1679 [34:05<00:00,  1.22s/it]


In [ ]:
df = pd.DataFrame(nogada)
df

,사건번호,내용,결과,날짜,관할위원회
0,2022부해1317,가. 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,01
1,2022부해1641,가. 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,01
2,2022부해1608,"① 근로자는 구인공고를 보고 입사지원을 하였고, 구인공고에는 “1개월 단기 당직의 ...",기각,2022-09-01,01
3,2022부해1595,① 근로자는 관리소장이 ‘지시를 거부하는 근로자는 필요 없다. 다른 사람을 구할 테...,기각,2022-08-31,01
4,2022부해1370,가. 근로계약의 법적 성격 ① 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,01
...,...,...,...,...,...
16785,2011부해792,,전부인정,--,01
16786,2012부해425,,각하,--,01
16787,2012부해1556,,전부인정,--,01
16788,2012부해2240,,각하,--,01


크롤링한 데이터를 엑셀로 저장


In [ ]:
df.to_excel('서울노동위원회 부당해고.xlsx', index=False)